In [17]:
import pytz
from datetime import datetime, timedelta
from gym_mtsim import MtSimulator, OrderType, Timeframe, FOREX_DATA_PATH
import matplotlib.pyplot as plt

selected_symbol = ['EURUSD', 'GBPCAD', 'GBPUSD', 'USDCAD', 'USDCHF', 'GBPJPY', 'USDJPY']
file_name = '4H_test_final.pkl'

# Create sim
sim = MtSimulator(
    unit='USD',
    balance=10000.,
    leverage=100.,
    stop_out_level=0.2,
    hedge=False,
)

# Load or create new file
if not sim.load_symbols(file_name):
    sim.download_data(
        symbols = selected_symbol,
        time_range=(
            datetime(2022, 1, 16, tzinfo=pytz.UTC),
            datetime(2022, 3, 20, tzinfo=pytz.UTC)
        ),
        timeframe=Timeframe.H4
        #timeframe=Timeframe.D1
    )

# Define df

In [18]:
df = sim.symbols_data

# Change column name to lower case for finTA

In [19]:
for i in selected_symbol:
    df[i] = df[i].rename(columns=str.lower)

df

{'EURUSD':                               open    close      low     high  volume
 Time                                                                 
 2022-01-17 00:00:00+00:00  1.14117  1.14050  1.14002  1.14159    3922
 2022-01-17 04:00:00+00:00  1.14051  1.14188  1.14038  1.14224    2958
 2022-01-17 08:00:00+00:00  1.14189  1.14240  1.14120  1.14342    7318
 2022-01-17 12:00:00+00:00  1.14240  1.14018  1.13915  1.14266    9574
 2022-01-17 16:00:00+00:00  1.14017  1.14118  1.13956  1.14159    9234
 ...                            ...      ...      ...      ...     ...
 2022-03-18 04:00:00+00:00  1.10878  1.10841  1.10753  1.10907    8131
 2022-03-18 08:00:00+00:00  1.10841  1.10573  1.10532  1.10944   16801
 2022-03-18 12:00:00+00:00  1.10573  1.10254  1.10031  1.10604   22362
 2022-03-18 16:00:00+00:00  1.10254  1.10586  1.10243  1.10720   25794
 2022-03-18 20:00:00+00:00  1.10584  1.10503  1.10422  1.10591    8197
 
 [270 rows x 5 columns],
 'GBPCAD':                              

# Change volume data type to float

In [20]:
for symbol in range(0,len(selected_symbol)):
    #print(type(sim.symbols_data[selected_symbol[symbol]]))
    df[selected_symbol[symbol]]['volume'] = df[selected_symbol[symbol]]['volume'].astype(float)
print(df[selected_symbol[2]].dtypes)

open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object


## Add indicator for each selected_symbol

In [21]:
from finta import TA


for symbol in range(0,len(selected_symbol)):
    df[selected_symbol[symbol]]['EFI'] = TA.EFI(df[selected_symbol[symbol]])
    df[selected_symbol[symbol]]['ATR'] = TA.ATR(df[selected_symbol[symbol]])
    df[selected_symbol[symbol]]['ADX'] = TA.ADX(df[selected_symbol[symbol]])
    df[selected_symbol[symbol]]['STOCHRSI'] = TA.STOCHRSI(df[selected_symbol[symbol]])

df[selected_symbol[0]].head()

,open,close,low,high,volume,EFI,ATR,ADX,STOCHRSI
Time,,,,,,,,,
2022-01-17 00:00:00+00:00,1.14117,1.14050,1.14002,1.14159,3922.0,NaN,NaN,NaN,NaN
2022-01-17 04:00:00+00:00,1.14051,1.14188,1.14038,1.14224,2958.0,4.082040,NaN,NaN,NaN
2022-01-17 08:00:00+00:00,1.14189,1.14240,1.14120,1.14342,7318.0,3.933058,NaN,NaN,NaN
2022-01-17 12:00:00+00:00,1.14240,1.14018,1.13915,1.14266,9574.0,-5.784891,NaN,NaN,NaN
2022-01-17 16:00:00+00:00,1.14017,1.14118,1.13956,1.14159,9234.0,-1.122919,NaN,NaN,NaN


In [22]:
# Fill NA

for symbol in range(0,len(selected_symbol)):
    df[selected_symbol[symbol]].fillna(0, inplace=True)

df[selected_symbol[0]].head()

,open,close,low,high,volume,EFI,ATR,ADX,STOCHRSI
Time,,,,,,,,,
2022-01-17 00:00:00+00:00,1.14117,1.14050,1.14002,1.14159,3922.0,0.000000,0.0,0.0,0.0
2022-01-17 04:00:00+00:00,1.14051,1.14188,1.14038,1.14224,2958.0,4.082040,0.0,0.0,0.0
2022-01-17 08:00:00+00:00,1.14189,1.14240,1.14120,1.14342,7318.0,3.933058,0.0,0.0,0.0
2022-01-17 12:00:00+00:00,1.14240,1.14018,1.13915,1.14266,9574.0,-5.784891,0.0,0.0,0.0
2022-01-17 16:00:00+00:00,1.14017,1.14118,1.13956,1.14159,9234.0,-1.122919,0.0,0.0,0.0


In [23]:
indicator_list = ['EFI','ATR','ADX','STOCHRSI']

for symbol in range(0,len(selected_symbol)):
    print('------')
    print('Corr for', selected_symbol[symbol])
    print('------')
    print(df[selected_symbol[symbol]][indicator_list].corr())

------
Corr for EURUSD
------
               EFI       ATR       ADX  STOCHRSI
EFI       1.000000 -0.008266 -0.005649  0.442753
ATR      -0.008266  1.000000  0.133011  0.197005
ADX      -0.005649  0.133011  1.000000 -0.162521
STOCHRSI  0.442753  0.197005 -0.162521  1.000000
------
Corr for GBPCAD
------
               EFI       ATR       ADX  STOCHRSI
EFI       1.000000 -0.075339 -0.271880  0.396256
ATR      -0.075339  1.000000  0.434647  0.340597
ADX      -0.271880  0.434647  1.000000 -0.260557
STOCHRSI  0.396256  0.340597 -0.260557  1.000000
------
Corr for GBPUSD
------
               EFI       ATR       ADX  STOCHRSI
EFI       1.000000 -0.247502 -0.193992  0.408390
ATR      -0.247502  1.000000  0.307600  0.205487
ADX      -0.193992  0.307600  1.000000 -0.291898
STOCHRSI  0.408390  0.205487 -0.291898  1.000000
------
Corr for USDCAD
------
               EFI       ATR       ADX  STOCHRSI
EFI       1.000000  0.202249  0.297295  0.291210
ATR       0.202249  1.000000  0.452086  0.74116

In [24]:
for symbol in range(0,len(selected_symbol)):
    df[selected_symbol[symbol]] = df[selected_symbol[symbol]][['open', 'close', 'low', 'high', 'volume'] + indicator_list]
    print(df[selected_symbol[symbol]].head())

                              open    close      low     high  volume  \
Time                                                                    
2022-01-17 00:00:00+00:00  1.14117  1.14050  1.14002  1.14159  3922.0   
2022-01-17 04:00:00+00:00  1.14051  1.14188  1.14038  1.14224  2958.0   
2022-01-17 08:00:00+00:00  1.14189  1.14240  1.14120  1.14342  7318.0   
2022-01-17 12:00:00+00:00  1.14240  1.14018  1.13915  1.14266  9574.0   
2022-01-17 16:00:00+00:00  1.14017  1.14118  1.13956  1.14159  9234.0   

                                EFI  ATR  ADX  STOCHRSI  
Time                                                     
2022-01-17 00:00:00+00:00  0.000000  0.0  0.0       0.0  
2022-01-17 04:00:00+00:00  4.082040  0.0  0.0       0.0  
2022-01-17 08:00:00+00:00  3.933058  0.0  0.0       0.0  
2022-01-17 12:00:00+00:00 -5.784891  0.0  0.0       0.0  
2022-01-17 16:00:00+00:00 -1.122919  0.0  0.0       0.0  
                              open    close      low     high   volume  \
Time    

In [25]:
df[selected_symbol[0]].shape

(270, 9)

In [26]:
df

{'EURUSD':                               open    close      low     high   volume  \
 Time                                                                     
 2022-01-17 00:00:00+00:00  1.14117  1.14050  1.14002  1.14159   3922.0   
 2022-01-17 04:00:00+00:00  1.14051  1.14188  1.14038  1.14224   2958.0   
 2022-01-17 08:00:00+00:00  1.14189  1.14240  1.14120  1.14342   7318.0   
 2022-01-17 12:00:00+00:00  1.14240  1.14018  1.13915  1.14266   9574.0   
 2022-01-17 16:00:00+00:00  1.14017  1.14118  1.13956  1.14159   9234.0   
 ...                            ...      ...      ...      ...      ...   
 2022-03-18 04:00:00+00:00  1.10878  1.10841  1.10753  1.10907   8131.0   
 2022-03-18 08:00:00+00:00  1.10841  1.10573  1.10532  1.10944  16801.0   
 2022-03-18 12:00:00+00:00  1.10573  1.10254  1.10031  1.10604  22362.0   
 2022-03-18 16:00:00+00:00  1.10254  1.10586  1.10243  1.10720  25794.0   
 2022-03-18 20:00:00+00:00  1.10584  1.10503  1.10422  1.10591   8197.0   
 
             

# Change column name to match with observation space

In [27]:
# Change column to lower case
for i in selected_symbol:
    #sim.symbols_data[i] = sim.symbols_data[i].rename({'close':'Close'})
    df[i].rename({'close':'Close', 'open':'Open', 'low':'Low','high':'High','volume':'Volume'}, axis=1, inplace=True)

df

{'EURUSD':                               Open    Close      Low     High   Volume  \
 Time                                                                     
 2022-01-17 00:00:00+00:00  1.14117  1.14050  1.14002  1.14159   3922.0   
 2022-01-17 04:00:00+00:00  1.14051  1.14188  1.14038  1.14224   2958.0   
 2022-01-17 08:00:00+00:00  1.14189  1.14240  1.14120  1.14342   7318.0   
 2022-01-17 12:00:00+00:00  1.14240  1.14018  1.13915  1.14266   9574.0   
 2022-01-17 16:00:00+00:00  1.14017  1.14118  1.13956  1.14159   9234.0   
 ...                            ...      ...      ...      ...      ...   
 2022-03-18 04:00:00+00:00  1.10878  1.10841  1.10753  1.10907   8131.0   
 2022-03-18 08:00:00+00:00  1.10841  1.10573  1.10532  1.10944  16801.0   
 2022-03-18 12:00:00+00:00  1.10573  1.10254  1.10031  1.10604  22362.0   
 2022-03-18 16:00:00+00:00  1.10254  1.10586  1.10243  1.10720  25794.0   
 2022-03-18 20:00:00+00:00  1.10584  1.10503  1.10422  1.10591   8197.0   
 
             

# Check suitable window size

In [28]:
df['EURUSD'].head()

,Open,Close,Low,High,Volume,EFI,ATR,ADX,STOCHRSI
Time,,,,,,,,,
2022-01-17 00:00:00+00:00,1.14117,1.14050,1.14002,1.14159,3922.0,0.000000,0.0,0.0,0.0
2022-01-17 04:00:00+00:00,1.14051,1.14188,1.14038,1.14224,2958.0,4.082040,0.0,0.0,0.0
2022-01-17 08:00:00+00:00,1.14189,1.14240,1.14120,1.14342,7318.0,3.933058,0.0,0.0,0.0
2022-01-17 12:00:00+00:00,1.14240,1.14018,1.13915,1.14266,9574.0,-5.784891,0.0,0.0,0.0
2022-01-17 16:00:00+00:00,1.14017,1.14118,1.13956,1.14159,9234.0,-1.122919,0.0,0.0,0.0


In [29]:
# start => window size should start at 15 to prevent 0 data
start = 13
end = 20

for i in selected_symbol:
    print("--------")
    print(i)
    print("--------")
    print(df[i][['ADX','STOCHRSI','ATR']].iloc[start-1:end])

--------
EURUSD
--------
                             ADX  STOCHRSI       ATR
Time                                                
2022-01-19 00:00:00+00:00    0.0  0.000000  0.000000
2022-01-19 04:00:00+00:00  100.0  0.000000  0.002404
2022-01-19 08:00:00+00:00  100.0  0.348258  0.002444
2022-01-19 12:00:00+00:00  100.0  0.291548  0.002439
2022-01-19 16:00:00+00:00  100.0  0.234838  0.002430
2022-01-19 20:00:00+00:00  100.0  0.226731  0.002276
2022-01-20 00:00:00+00:00  100.0  0.217041  0.002317
2022-01-20 04:00:00+00:00  100.0  0.209515  0.002364
--------
GBPCAD
--------
                                  ADX  STOCHRSI       ATR
Time                                                     
2022-01-19 00:00:00+00:00    0.000000  0.000000  0.000000
2022-01-19 04:00:00+00:00  100.000000  0.000000  0.003668
2022-01-19 08:00:00+00:00   79.276033  0.173436  0.003802
2022-01-19 12:00:00+00:00   69.942384  0.123401  0.003988
2022-01-19 16:00:00+00:00   74.079698  0.146173  0.004044
2022-01-19 20:

In [30]:
# Create a copy of open, close

for i in selected_symbol:
    df[i]['open'] = df[i]['Open']
    df[i]['close'] = df[i]['Close']
    df[i]['low'] = df[i]['Low']
    df[i]['high'] = df[i]['High']

df['EURUSD'].head()

,Open,Close,Low,High,Volume,EFI,ATR,ADX,STOCHRSI,open,close,low,high
Time,,,,,,,,,,,,,
2022-01-17 00:00:00+00:00,1.14117,1.14050,1.14002,1.14159,3922.0,0.000000,0.0,0.0,0.0,1.14117,1.14050,1.14002,1.14159
2022-01-17 04:00:00+00:00,1.14051,1.14188,1.14038,1.14224,2958.0,4.082040,0.0,0.0,0.0,1.14051,1.14188,1.14038,1.14224
2022-01-17 08:00:00+00:00,1.14189,1.14240,1.14120,1.14342,7318.0,3.933058,0.0,0.0,0.0,1.14189,1.14240,1.14120,1.14342
2022-01-17 12:00:00+00:00,1.14240,1.14018,1.13915,1.14266,9574.0,-5.784891,0.0,0.0,0.0,1.14240,1.14018,1.13915,1.14266
2022-01-17 16:00:00+00:00,1.14017,1.14118,1.13956,1.14159,9234.0,-1.122919,0.0,0.0,0.0,1.14017,1.14118,1.13956,1.14159


In [31]:
from sklearn.preprocessing import MinMaxScaler
import joblib
import numpy as np

# Volume
#columns = ['Open','Close','Low','High','Volume','EFI','ADX','ATR','STOCHRSI']
columns = ['open','close','low','high','Volume','EFI','ADX','ATR','STOCHRSI']


for i in selected_symbol:
    scaler_filename = "scaler/scaler_" + i + "_4H"
    print("Load scaler from",scaler_filename)
    scaler = joblib.load(scaler_filename) 
    df[i][columns] = scaler.transform(df[i][columns])
    display(df[i])


Load scaler from scaler/scaler_EURUSD_4H


,Open,Close,Low,High,Volume,EFI,ATR,ADX,STOCHRSI,open,close,low,high
Time,,,,,,,,,,,,,
2022-01-17 00:00:00+00:00,1.14117,1.14050,1.14002,1.14159,0.057644,0.673642,0.000000,0.000000,0.000000,0.202988,0.196635,0.201962,0.198549
2022-01-17 04:00:00+00:00,1.14051,1.14188,1.14038,1.14224,0.033325,0.724579,0.000000,0.000000,0.000000,0.196786,0.209606,0.205358,0.204674
2022-01-17 08:00:00+00:00,1.14189,1.14240,1.14120,1.14342,0.143315,0.722720,0.000000,0.000000,0.000000,0.209755,0.214494,0.213093,0.215793
2022-01-17 12:00:00+00:00,1.14240,1.14018,1.13915,1.14266,0.200227,0.601458,0.000000,0.000000,0.000000,0.214548,0.193627,0.193755,0.208632
2022-01-17 16:00:00+00:00,1.14017,1.14118,1.13956,1.14159,0.191650,0.659630,0.000000,0.000000,0.000000,0.193591,0.203027,0.197623,0.198549
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-18 04:00:00+00:00,1.10878,1.10841,1.10753,1.10907,0.163824,0.883118,1.058637,0.295549,0.532238,-0.101400,-0.104991,-0.104518,-0.107897
2022-03-18 08:00:00+00:00,1.10841,1.10573,1.10532,1.10944,0.382543,0.772929,1.087778,0.294007,0.537860,-0.104877,-0.130181,-0.125366,-0.104410
2022-03-18 12:00:00+00:00,1.10573,1.10254,1.10031,1.10604,0.522830,0.631584,1.165666,0.276910,0.537049,-0.130063,-0.160165,-0.172625,-0.136449


Load scaler from scaler/scaler_GBPCAD_4H


,Open,Close,Low,High,Volume,EFI,ATR,ADX,STOCHRSI,open,close,low,high
Time,,,,,,,,,,,,,
2022-01-17 00:00:00+00:00,1.71280,1.71487,1.71280,1.71622,0.128014,0.465538,0.000000,0.000000,0.000000,0.493380,0.516731,0.531155,0.488125
2022-01-17 04:00:00+00:00,1.71485,1.71494,1.71439,1.71583,0.093770,0.468751,0.000000,0.000000,0.000000,0.515997,0.517504,0.548415,0.483818
2022-01-17 08:00:00+00:00,1.71494,1.71137,1.71042,1.71551,0.257354,0.251820,0.000000,0.000000,0.000000,0.516990,0.478078,0.505319,0.480283
2022-01-17 12:00:00+00:00,1.71136,1.71038,1.70912,1.71168,0.266372,0.290126,0.000000,0.000000,0.000000,0.477493,0.467145,0.491207,0.437976
2022-01-17 16:00:00+00:00,1.71038,1.70792,1.70663,1.71102,0.261488,0.257816,0.000000,0.000000,0.000000,0.466681,0.439978,0.464177,0.430686
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-18 04:00:00+00:00,1.66105,1.65929,1.65899,1.66140,0.116132,0.390053,0.721791,0.250270,0.451533,-0.077560,-0.097073,-0.052974,-0.117420
2022-03-18 08:00:00+00:00,1.65931,1.65809,1.65613,1.65957,0.299327,0.378740,0.730846,0.256809,0.449097,-0.096756,-0.110326,-0.084021,-0.137634
2022-03-18 12:00:00+00:00,1.65810,1.65752,1.65366,1.65920,0.358027,0.378714,0.757313,0.266914,0.447233,-0.110106,-0.116621,-0.110834,-0.141721


Load scaler from scaler/scaler_GBPUSD_4H


...

,Open,Close,Low,High,Volume,EFI,ATR,ADX,STOCHRSI,open,close,low,high
Time,,,,,,,,,,,,,
2022-01-17 00:00:00+00:00,0.91219,0.91551,0.91219,0.91586,0.041451,0.450312,0.000000,0.000000,0.000000,0.442020,0.520379,0.455667,0.498601
2022-01-17 04:00:00+00:00,0.91551,0.91440,0.91406,0.91554,0.030249,0.411386,0.000000,0.000000,0.000000,0.520749,0.494076,0.499185,0.491142
2022-01-17 08:00:00+00:00,0.91435,0.91356,0.91219,0.91492,0.220277,0.377070,0.000000,0.000000,0.000000,0.493242,0.474171,0.455667,0.476690
2022-01-17 12:00:00+00:00,0.91356,0.91406,0.91281,0.91560,0.291208,0.435179,0.000000,0.000000,0.000000,0.474508,0.486019,0.470095,0.492541
2022-01-17 16:00:00+00:00,0.91406,0.91405,0.91315,0.91460,0.209762,0.439512,0.000000,0.000000,0.000000,0.486365,0.485782,0.478008,0.469231
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-18 04:00:00+00:00,0.93642,0.93623,0.93597,0.93716,0.115739,0.369412,0.807875,0.319787,0.919613,1.016599,1.011374,1.009076,0.995105
2022-03-18 08:00:00+00:00,0.93624,0.93438,0.93420,0.93825,0.303138,0.338629,0.857143,0.299297,0.892416,1.012331,0.967536,0.967885,1.020513
2022-03-18 12:00:00+00:00,0.93436,0.93580,0.93329,0.93730,0.421466,0.398090,0.920429,0.284031,0.871422,0.967750,1.001185,0.946707,0.998368


Load scaler from scaler/scaler_GBPJPY_4H


,Open,Close,Low,High,Volume,EFI,ATR,ADX,STOCHRSI,open,close,low,high
Time,,,,,,,,,,,,,
2022-01-17 00:00:00+00:00,155.926,156.452,155.926,156.492,0.169723,0.649772,0.000000,0.000000,0.000000,0.758948,0.815574,0.785835,0.807331
2022-01-17 04:00:00+00:00,156.456,156.551,156.377,156.582,0.115235,0.689522,0.000000,0.000000,0.000000,0.816085,0.826251,0.833298,0.817389
2022-01-17 08:00:00+00:00,156.551,156.626,156.379,156.646,0.207222,0.695972,0.000000,0.000000,0.000000,0.826326,0.834340,0.833509,0.824542
2022-01-17 12:00:00+00:00,156.626,156.359,156.236,156.709,0.237423,0.597349,0.000000,0.000000,0.000000,0.834411,0.805544,0.818459,0.831582
2022-01-17 16:00:00+00:00,156.359,156.465,156.275,156.508,0.206401,0.636230,0.000000,0.000000,0.000000,0.805627,0.816976,0.822564,0.809119
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-18 04:00:00+00:00,156.237,156.317,156.170,156.344,0.164614,0.797316,0.638502,0.454597,0.681686,0.792475,0.801014,0.811513,0.790791
2022-03-18 08:00:00+00:00,156.319,156.256,156.025,156.554,0.340413,0.766592,0.664194,0.459001,0.688097,0.801315,0.794435,0.796253,0.814260
2022-03-18 12:00:00+00:00,156.256,156.527,156.196,156.599,0.403788,0.800447,0.683373,0.463586,0.696636,0.794524,0.823663,0.814250,0.819289


Load scaler from scaler/scaler_USDJPY_4H


,Open,Close,Low,High,Volume,EFI,ATR,ADX,STOCHRSI,open,close,low,high
Time,,,,,,,,,,,,,
2022-01-17 00:00:00+00:00,114.117,114.468,114.117,114.514,0.152695,0.757564,0.000000,0.000000,0.000000,0.754887,0.794992,0.772093,0.784041
2022-01-17 04:00:00+00:00,114.473,114.491,114.386,114.496,0.071119,0.768333,0.000000,0.000000,0.000000,0.795587,0.797622,0.803372,0.781923
2022-01-17 08:00:00+00:00,114.491,114.501,114.309,114.547,0.133044,0.766337,0.000000,0.000000,0.000000,0.797645,0.798765,0.794419,0.787925
2022-01-17 12:00:00+00:00,114.500,114.535,114.446,114.598,0.086805,0.769886,0.000000,0.000000,0.000000,0.798674,0.802653,0.810349,0.793927
2022-01-17 16:00:00+00:00,114.535,114.621,114.525,114.649,0.073349,0.778790,0.000000,0.000000,0.000000,0.802675,0.812486,0.819535,0.799929
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-18 04:00:00+00:00,118.712,118.753,118.701,118.800,0.108908,0.794216,0.485624,0.482280,0.650251,1.280210,1.284930,1.305116,1.288455
2022-03-18 08:00:00+00:00,118.753,118.929,118.752,119.007,0.259026,0.818914,0.494490,0.483939,0.646974,1.284898,1.305054,1.311047,1.312816
2022-03-18 12:00:00+00:00,118.928,119.205,118.913,119.397,0.340280,0.869405,0.541609,0.492098,0.651063,1.304905,1.336611,1.329767,1.358715


In [33]:
#Save data
sim.save_symbols(file_name)